# A* Extension
This notebook provides an interactive interface to visualize the A* extension for a 2-DoF environment. It allows you to select different benchmarks and adjust the parameters.

## A* Configuration
First, we need to create the A* configuration with the newly introduced datafields for every extension:


In [ ]:
aStar_config = dict()
aStar_config["w"] = 0.5  # weight for the heuristic
aStar_config["heuristic"]  = "euclidean"  # heuristic function to use
aStar_config["reopen"] = False  # whether to allow reopening of nodes
aStar_config["dof"] = 2  # number of degrees of freedom
aStar_config["check_connection"] = False  # whether to check for collisions between nodes
aStar_config["lazy_check_connection"] = True  # only check the connection when node is expanded. We recommend to keep this as True (performance)
aStar_config["discretization"] = [10, 10]  # number of discretization steps per dof
aStar_config["benchmark"] = [0]  # benchmark to use, can be a list of indices from the benchmark list


Next, we set up the interactive widgets to set the parameters for the A* algorithm. The widgets will allow you to select a benchmark, set the discretization values and weights, and toggle options like reopening and line collision checks.

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from evaluation.two_dof.IPTestSuite_2DoF import benchList
from evaluation.two_dof.PlotEnvironments import visualizeBenchmark

slider_layout = widgets.Layout(width='400px')
slider_style = {'description_width': '150px'}

benchmark_slider = widgets.IntSlider(value=0, min=0, max=len(benchList)-1, step=1, description='Benchmark:', layout=slider_layout, style=slider_style)
discretizationX_slider = widgets.SelectionSlider(options=[10, 20, 50, 100, 200], value=10, description='X Discretization:', layout=slider_layout, style=slider_style)
discretizationY_slider = widgets.SelectionSlider(options=[10, 20, 50, 100, 200], value=10, description='Y Discretization:', layout=slider_layout, style=slider_style)
weight_slider = widgets.FloatSlider(value=0.5, min=0.5, max=1.0, step=0.01, description='Weight:', layout=slider_layout, style=slider_style)
reopening_slider = widgets.SelectionSlider(options=[False, True], value=False, description='Reopening:', layout=slider_layout, style=slider_style)
lineCollision_slider = widgets.SelectionSlider(options=[False, True], value=False, description='Line Collision:', layout=slider_layout, style=slider_style)

output = widgets.Output()  # output widget to capture the plots

def update_plot():
    with output:
        output.clear_output(wait=True)
        fig, ax = visualizeBenchmark(
            benchList[benchmark_slider.value],
            discretizationX=discretizationX_slider.value,
            discretizationY=discretizationY_slider.value
        )
        plt.show()

def update_a_star_config(change=None):
    aStar_config["benchmark"] = [benchmark_slider.value]
    aStar_config["discretization"] = [discretizationX_slider.value, discretizationY_slider.value]
    aStar_config["w"] = weight_slider.value
    aStar_config["reopen"] = reopening_slider.value
    aStar_config["check_connection"] = lineCollision_slider.value


update_plot()  # Initial plot

def on_any_slider_change(change):
    update_plot()
    update_a_star_config()

benchmark_slider.observe(on_any_slider_change, names='value')
discretizationX_slider.observe(on_any_slider_change, names='value')
discretizationY_slider.observe(on_any_slider_change, names='value')
weight_slider.observe(update_a_star_config, names='value')
lineCollision_slider.observe(update_a_star_config, names='value')
reopening_slider.observe(update_a_star_config, names='value')

display(benchmark_slider, discretizationX_slider, discretizationY_slider, weight_slider, lineCollision_slider, reopening_slider, output)

## A* Search Execution
Now that we have set up the configuration, we can execute the A* search algorithm using the selected benchmark and parameters. We provide a progress bar on how much of the search space has been explored (so it does not have to reach 100%).
This will also output the time taken for the planning process.

The `evaluate` function will run the A* algorithm with the specified configuration and benchmark, and it will return statistics, the solution path, deltas for visualization, and the A* instance itself. If `dump=True`, it will save the results to a directory named after the configuration and benchmark, so we do not have to run the search again for visualization.

In [ ]:
import sys, os, time
sys.path.append(os.path.join(os.getcwd(), 'core'))
sys.path.append(os.path.join(os.getcwd(), 'evaluation/two_dof'))

from core.IPAStarExtended import AStar
from Benchmarking import get_config_dir_name, evaluate

start_time = time.time()
stats, solution, deltas, astar = evaluate(config=aStar_config, benchmark=benchList[aStar_config["benchmark"][0]], dump=True)
print(f"Planning took {time.time() - start_time:.2f} seconds")


## Step-by-Step Visualization of the search process

Next, we can use the interactive plot to visualize the A* search. We render all frames at once using OpenCV to avoid the overhead of rendering each frame individually, which can be slow in Jupyter notebooks. This allows us to see the entire search process in one go.

In [ ]:
from Animate import animate
from IPython.display import display, Image

# get the directory of the benchmarking results to load them
config_dir = get_config_dir_name(aStar_config, benchList[aStar_config["benchmark"][0]].name)

animation_dir = animate(config_dir)  # animate the A* search and save frames to a directory

def get_animation_images(animation_dir):
    # Get all step_*.jpg files and sort them numerically
    image_files = [f for f in os.listdir(animation_dir) if f.startswith('step_') and f.endswith('.jpg')]
    image_files.sort(key=lambda x: int(os.path.basename(x).split('_')[1].split('.')[0]))
    return image_files

animation_images = get_animation_images(animation_dir)

# Create animation slider
animation_slider = widgets.IntSlider(value=0, min=0, max=len(animation_images)-1 if animation_images else 0, step=1, description='Animation Frame:',layout=slider_layout, style=slider_style)

image_output = widgets.Output()

def update_animation_image(change=None):
    if not animation_images:
        return

    with image_output:
        image_output.clear_output(wait=True)
        img_path = animation_images[animation_slider.value]
        display(Image(filename=animation_dir + '/' + img_path, width=600))

update_animation_image()

animation_slider.observe(update_animation_image, names='value')
display(animation_slider, image_output)